<a href="https://colab.research.google.com/github/sameepshrestha/Autocorrect-/blob/main/Autocorrect_max_probability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np 
import pandas as pd 
import nltk 
import matplotlib.pyplot as plt
import re 
from collections import Counter

#to break the paragraph into each sentence 

In [17]:
class Sentinizer():
    def __init__(self,input_text,splitter=['.','?','!',':'],delimiter_token='<split>'):
        # self.sentences = []
        self.raw = input_text
        self._split_characters = splitter
        self._delimiter_token = delimiter_token
        self._index = 0
        self._sentencize()
    def _sentencize(self):
        work_sentence = self.raw
        for characters in self._split_characters:
            work_sentence=work_sentence.replace(characters,characters+''+self._delimiter_token)
        self.sentences= [x for x in work_sentence.split(self._delimiter_token) if x!='']
    def __iter__(self):
        return self
    def __next__(self):
        if self._index<len(self.sentences):
            result= self.sentences[self._index]
            self._index+=1
            return result
        else:
            raise StopIteration

# to tokenize a sentence 

In [65]:
class Tokenizer():
    import string
    
    def __init__(self, sentence , token_boundries=[' ','-','.'],delimiter_token='<SPLIT>',punctuations = string.punctuation ):
        self.words = []
        self.sentence = sentence.lower()
        self._token_boundries = token_boundries
        self._index = 0
        self._punctuations = punctuations
        self._delimiter_token = delimiter_token
        self._tokenize()
    def _tokenize(self):
        work_sentence = self.sentence
        token = self._delimiter_token
        for characters in self._punctuations:
            work_sentence = work_sentence.replace(characters , ''+ characters + "")
        for characters in self._token_boundries:
            work_sentence = work_sentence.replace(characters,token)
        self.words= [x for x in work_sentence.split(token) if x!='']
    def __iter__(self):
        return self 
    def __next__(self):
        if self._index<len(self.words):
            result= self.words[self._index]
            self._index+=1
            return result 
        else:
            self._index=0
            raise StopIteration

In [ ]:
# Probable replacing words for the incorrect word in a sentence 

In [20]:
class Edit():
    def __init__(self, words, alphabet='abcdefghijklmnopqrstuvwxyz'):
        self.swords=[]
        self.word = words 
        self. alphabet = alphabet 
        self.splits()
        self.replace()
        self.delete()
        self.switch()
        self.insert()
        self.twords()
        # self.replace()
    def splits(self):
        word = self.word
        self.split = [(word[0:i],word[i:]) for i in range(len(word)+1)]
    def replace(self):
        wrong_word = self.word
        splits = self.split
        alphabet=self.alphabet
        self.rwords = list(set([L +alphabet[j]+ K[1:] for L, K in self.split for j in range(len(alphabet))]))
    def delete(self):
        wrong_word = self.word
        splits = self.split
        self.dwords = list(set([L + K[1:] for L, K in self.split if K!=0]))
    def switch(self):
        wrong_word = self.word
        splits = self.split
        self.swords = list(set([L+R[1]+R[0]+R[2:] for L,R in splits if len(R)>=2]))
    def insert(self):
        wrong_word = self.word
        splits = self.split
        alphabet=self.alphabet
        self.iwords = list(set([L+alphabet[j]+R for L, R in splits for j in range (len(alphabet))]))
    def twords(self):
        self.total_words= list(set(self.rwords+self.dwords+self.swords+self.iwords))    

''' main is the vocab containing the words and the number they have occured in a vocab so that  
we know the word with their probability though this approch  seems less obvious, for better accurate 
we can also evaluate the probabaility of the simultaneous occuring of the word''' 

#Generating the vocab from the given text file 

In [22]:
# generator for reading only word per step for large vocab
def words_generator(file_name):
    for line in open(file_name):
        for word in re.findall('\w+',line):
            yield word.lower()

In [43]:
file_name = '/content/shakespere.txt'
vocab = {}
count = 0
for word in (words_generator(file_name)):
    vocab[word] = vocab.get(word,0)+1
    count+=1
vocab = { k: v/count for k, v in vocab.items()}#vocab with probabilities as values

# Determing all the possible replacement word for the incorrect word 

In [100]:
# n_corrections for allowing correction of amount of words in the incorrect word
def correct_words(word, n_corrections=1):
    probable_words = Edit(word)
    total_correct_words = probable_words.total_words
    for i in range(n_corrections-1):
        for words in probable_words.total_words:
            probab = Edit(words)
            total_correct_words= total_correct_words+probab.total_words
            #need to check if the word after replace,switch etc exist in vocab as only those in vocab
            # are the possible solutions
    return ([word for word in set(total_correct_words) if word in vocab.keys()])
            
    

In [102]:
correct_word = correct_words('whre',n_corrections=2)
print(correct_word[:10])
correct_word = correct_words('whre',n_corrections=1)
print(correct_word[:10])

['she', 'fore', 'wife', 'fare', 'white', 'whores', 'who', 'work', 'where', 'bore']
['whoe', 'where', 'wore', 'were']


# Bringing it all together
In this method we will use simple probabilistic method for single sentence.    
we could build a probability of two words coming together to determine the word.    
we could also use minmum distance for the correction which would be done later.

In [117]:
def best_probability(words):
    prbab = np.nan
    for word in words:
        if vocab[word] > prbab :
            probab = vocab[word]
            probab_words = word 
    return word

def Autocorrect(sentence):
    words = Tokenizer(sentence)
    incorrect_words = [word for word in words if word not in vocab.keys()]
    all_Words = {word:correct_words(word) for word in incorrect_words}
    corrected_words = { i : best_probability(j) for i,j in all_Words.items()}
    return corrected_words
Autocorrect('This is wher it al starts')

where 0.0017532733987391353
when 0.003674413399485209
her 0.005259820196217406
ay 0.0005035998060208155
a 0.01411944641325027
all 0.004812175924198903
ail 1.865184466743761e-05
ah 0.00014921475733950087
am 0.002331480583429701
tal 1.865184466743761e-05
an 0.0017719252434065728
ll 0.0016600141754019473
at 0.0024433916514343267
alt 1.865184466743761e-05
as 0.0058939829149102846


{'al': 'as', 'wher': 'her'}

As you can see this is really bad, i mean really really bad so we will try minimum distance method in near future.